In [3]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re

In [4]:
# Test
# Test 2
# Test 3

URL = 'https://fbref.com/en/comps/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
# Club International Cups
club_int_comps = []

for j in soup.find_all('tbody')[0].find_all("tr", {"class": "gender-m"}):
    if (j.find('td', {'data-stat':'tier'}) != None):
        if (j.find('td', {'data-stat':'tier'}).text == '1st') | (j.find('td', {'data-stat':'tier'}).text == '2nd'):
            club_int_comps.append(j.find('a')['href'])

club_int_comps

['/en/comps/14/history/Copa-Libertadores-Seasons',
 '/en/comps/205/history/Copa-Sudamericana-Seasons',
 '/en/comps/8/history/UEFA-Champions-League-Seasons',
 '/en/comps/19/history/UEFA-Europa-League-Seasons']

In [6]:
# National Team Competitions
national_comps = []

for j in soup.find_all('tbody')[1].find_all("tr", {"class": "gender-m"}):
    if (j.find('td', {'data-stat':'tier'}) != None):
        if (j.find('td', {'data-stat':'tier'}).text == '1st') | (j.find('td', {'data-stat':'tier'}).text == '2nd'):
            national_comps.append(j.find('a')['href'])

national_comps

['/en/comps/1/history/World-Cup-Seasons',
 '/en/comps/656/history/Africa-Cup-of-Nations-Seasons',
 '/en/comps/681/history/CONCACAF-Gold-Cup-Seasons',
 '/en/comps/257/history/OFC-Nations-Cup-Seasons',
 '/en/comps/676/history/European-Championship-Seasons',
 '/en/comps/664/history/AFC-Asian-Cup-Seasons']

In [7]:
# Domestic Tier 1 Comps
dom_tier1_comps = []

for j in soup.find_all('tbody')[3].find_all("tr", {"class": "gender-m"}):
    if (j.find('td') != None):
        dom_tier1_comps.append(j.find('a')['href'])

dom_tier1_comps

['/en/comps/21/history/Primera-Division-Seasons',
 '/en/comps/65/history/A-League-Seasons',
 '/en/comps/56/history/Austrian-Bundesliga-Seasons',
 '/en/comps/37/history/Belgian-First-Division-A-Seasons',
 '/en/comps/74/history/Primera-Division-Seasons',
 '/en/comps/24/history/Serie-A-Seasons',
 '/en/comps/67/history/Bulgarian-First-League-Seasons',
 '/en/comps/211/history/Canadian-Premier-League-Seasons',
 '/en/comps/35/history/Primera-Division-Seasons',
 '/en/comps/62/history/Super-League-Seasons',
 '/en/comps/41/history/Primera-A-Seasons',
 '/en/comps/63/history/1-HNL-Seasons',
 '/en/comps/66/history/Czech-First-League-Seasons',
 '/en/comps/50/history/Superliga-Seasons',
 '/en/comps/58/history/Serie-A-Seasons',
 '/en/comps/9/history/Premier-League-Seasons',
 '/en/comps/12/history/La-Liga-Seasons',
 '/en/comps/43/history/Veikkausliiga-Seasons',
 '/en/comps/13/history/Ligue-1-Seasons',
 '/en/comps/20/history/Bundesliga-Seasons',
 '/en/comps/27/history/Super-League-Greece-Seasons',
 '/en

In [8]:
all_comps = club_int_comps + national_comps + dom_tier1_comps

len(all_comps)

58

In [9]:
all_comps[-1]

'/en/comps/105/history/Liga-FUTVE-Seasons'

In [10]:
def get_all_seasons(url):
    URL = 'https://fbref.com/' + url
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    url_list = []
    
    for row in soup.find_all('tr'):
        if row.find('th',{"scope":"row"}) != None:
            url_list.append((row.find('a')['href']))
    
    return url_list

In [11]:
all_seasons = []

for i in all_comps:
    league_seasons = get_all_seasons(i)
    all_seasons += league_seasons

In [12]:
len(all_seasons)

886

In [13]:
# keep jsut in case

# def get_all_player_urls(url_list):
    
#     final_player_urls = []

#     for i in url_list:
#         URL_test = 'https://fbref.com/'+ i
#         page = requests.get(URL_test)
#         soup = BeautifulSoup(page.content, 'html.parser')
#         td1 = soup.find(lambda t: t.text.strip()=='Standard Stats')
#         td2 = td1.find_next('a')
#         url_to_add = td2['href']
#         final_player_urls.append(url_to_add)

#     return final_player_urls

In [14]:
def get_all_player_urls(url_list):
    
    final_player_urls = []

    for i in url_list:
        final_pos = i.rindex('/')

        final_player_urls.append(i[:final_pos+1]+'stats/'+i[final_pos+1:])

    return final_player_urls

In [15]:
final_stats = get_all_player_urls(all_seasons)

In [16]:
final_stats[287]

'/en/comps/50/141/stats/2004-2005-Superliga-Stats'

In [17]:
all_standard_urls = get_all_player_urls(all_seasons)

In [18]:
def get_player_table(url):
    url = 'https://fbref.com/' + url
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    player_table = all_tables[2]

    return player_table

In [19]:
testing = get_player_table(all_standard_urls[297])

# testing

In [20]:
stats = ["player","nationality","position","squad","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]

In [21]:
def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)

    return df_player

In [27]:
get_frame(stats, testing).columns



Index(['player', 'nationality', 'position', 'squad', 'age', 'birth_year',
       'games', 'games_starts', 'minutes', 'goals', 'assists', 'pens_made',
       'pens_att', 'cards_yellow', 'cards_red', 'goals_per90', 'assists_per90',
       'goals_assists_per90', 'goals_pens_per90', 'goals_assists_pens_per90',
       'xg', 'npxg', 'xa', 'xg_per90', 'xa_per90', 'xg_xa_per90', 'npxg_per90',
       'npxg_xa_per90'],
      dtype='object')